# ***ECO482 Group Project: An American Disaster Story: Predicting the Economic Damage Inflicted by Tornados​***

In [55]:
#mounting/loading google drive (working on google colab)
from google.colab import drive
drive.mount('/content/drive')

#loading useful packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import re

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
# Step 2: Define the folder path (ensure correctness)
folder_path = "/content/drive/My Drive/GDToT/ECO482_Project/Data"

# Step 3: Use glob to get all CSV file paths
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))  # Assumes all files end in .csv

# Step 4: Load and concatenate all CSVs
df_list = [pd.read_csv(file) for file in csv_files]  # Read each CSV into a list
storm_data = pd.concat(df_list, ignore_index=True)  # Combine into one DataFrame


# Optionally, show first few rows
storm_data.head()

<ipython-input-57-effb9a252c8d>:8: DtypeWarning: Columns (29,34,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(file) for file in csv_files]  # Read each CSV into a list
<ipython-input-57-effb9a252c8d>:8: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(file) for file in csv_files]  # Read each CSV into a list
<ipython-input-57-effb9a252c8d>:8: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(file) for file in csv_files]  # Read each CSV into a list
<ipython-input-57-effb9a252c8d>:8: DtypeWarning: Columns (26,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(file) for file in csv_files]  # Read each CSV into a list
<ipython-input-57-effb9a252c8d>:8: DtypeWarning: Columns (26,28) have mixed types. Specify dtype option on impor

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,202405,23,1947,202405,23,1947,190907.0,1180619,OKLAHOMA,40.0,...,4.0,S,FRIENDSHIP,34.6380,-99.2167,34.6380,-99.2167,Two primary rounds of severe convection occurr...,MPing report.,CSV
1,202411,16,230,202411,18,1421,197838.0,1223377,OREGON,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A series of cold fronts the weekend of Nov. 16...,The Hog Pass SNOTEL reported an estimated 12 i...,CSV
2,202405,19,1839,202405,19,1902,190905.0,1184919,OKLAHOMA,40.0,...,5.0,N,CUSTER CITY,35.7100,-99.0010,35.7370,-98.8910,Significant severe weather occurred across por...,While the large multiple-vortex tornado was ap...,CSV
3,202405,23,2155,202405,23,2155,190907.0,1180805,OKLAHOMA,40.0,...,2.0,W,NINNEKAH,34.9501,-97.9523,34.9501,-97.9523,Two primary rounds of severe convection occurr...,Relayed image showing a portable outbuilding b...,CSV
4,202411,22,400,202411,22,1100,197326.0,1220680,NEW YORK,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,An occluded low pressure system brought heavy ...,A trained spotter reported 10.0 inches of wet ...,CSV


In [3]:
storm_data

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,202405,23,1947,202405,23,1947,190907.0,1180619,OKLAHOMA,40.0,...,4.0,S,FRIENDSHIP,34.6380,-99.2167,34.6380,-99.2167,Two primary rounds of severe convection occurr...,MPing report.,CSV
1,202411,16,230,202411,18,1421,197838.0,1223377,OREGON,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A series of cold fronts the weekend of Nov. 16...,The Hog Pass SNOTEL reported an estimated 12 i...,CSV
2,202405,19,1839,202405,19,1902,190905.0,1184919,OKLAHOMA,40.0,...,5.0,N,CUSTER CITY,35.7100,-99.0010,35.7370,-98.8910,Significant severe weather occurred across por...,While the large multiple-vortex tornado was ap...,CSV
3,202405,23,2155,202405,23,2155,190907.0,1180805,OKLAHOMA,40.0,...,2.0,W,NINNEKAH,34.9501,-97.9523,34.9501,-97.9523,Two primary rounds of severe convection occurr...,Relayed image showing a portable outbuilding b...,CSV
4,202411,22,400,202411,22,1100,197326.0,1220680,NEW YORK,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,An occluded low pressure system brought heavy ...,A trained spotter reported 10.0 inches of wet ...,CSV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1862526,198106,1,1710,198106,1,1710,NaN,10129236,TEXAS,48.0,...,0.0,NaN,NaN,34.6000,-100.6500,NaN,NaN,NaN,NaN,PUB
1862527,198106,1,1720,198106,1,1720,NaN,10129237,TEXAS,48.0,...,0.0,NaN,NaN,34.9000,-100.4200,NaN,NaN,NaN,NaN,PUB
1862528,198106,1,1730,198106,1,1730,NaN,10129238,TEXAS,48.0,...,0.0,NaN,NaN,34.8200,-100.0300,34.8200,-99.9300,NaN,NaN,PUB
1862529,198106,1,2040,198106,1,2040,NaN,10129239,TEXAS,48.0,...,0.0,NaN,NaN,34.1500,-99.2800,NaN,NaN,NaN,NaN,PUB


In [4]:
storm_data.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'],
      dtype='object')

In [5]:
storm_data["DURATION"] = (storm_data["END_YEARMONTH"] - storm_data["BEGIN_YEARMONTH"])*30 + storm_data["END_DAY"] - storm_data["BEGIN_DAY"]
storm_data['COUNTY/PARISH'] = (storm_data['CZ_TYPE'] == "C").astype(int)
storm_data['MARINE_ZONE'] = (storm_data['CZ_TYPE'] == "M").astype(int)
storm_data['NWS_PUBLIC_FORECAST_ZONE'] = (storm_data['CZ_TYPE'] == "Z").astype(int)

In [58]:
def convert_to_dollars(value):
    # Handle NaN or None
    if pd.isna(value):
        return 0.0

    # Convert to string, strip whitespace, uppercase
    val_str = str(value).strip().upper()

    # If empty after stripping, return 0
    if not val_str:
        return 0.0

    # Regex to capture: one or more digits and/or '.', plus an optional single letter K/M/B
    pattern = r'^([\d\.]+)([KMB]?)$'
    match = re.match(pattern, val_str)

    # If it doesn't match, try direct float conversion or fallback to 0
    if not match:
        try:
            return float(val_str)
        except ValueError:
            return 0.0

    # Extract the numeric part and optional suffix
    num_str, suffix = match.groups()

    # Convert numeric part to float
    try:
        num = float(num_str)
    except ValueError:
        return 0.0

    # Apply multiplier if suffix is present
    if suffix == 'K':
        return num * 1e3
    elif suffix == 'M':
        return num * 1e6
    elif suffix == 'B':
        return num * 1e9
    else:
        # No suffix -> raw number
        return num

# Apply conversion
storm_data['DAMAGE_PROPERTY'] = storm_data['DAMAGE_PROPERTY'].apply(convert_to_dollars)
storm_data['DAMAGE_CROPS'] = storm_data['DAMAGE_CROPS'].apply(convert_to_dollars)

# Compute NET_DAMAGE
storm_data['NET_DAMAGE'] = storm_data['DAMAGE_PROPERTY'] + storm_data['DAMAGE_CROPS']

# Check the first few rows to ensure conversion worked
print(storm_data[['DAMAGE_PROPERTY','DAMAGE_CROPS','NET_DAMAGE']].head(20))
print(storm_data[['DAMAGE_PROPERTY','DAMAGE_CROPS','NET_DAMAGE']].describe())


    DAMAGE_PROPERTY  DAMAGE_CROPS  NET_DAMAGE
0               0.0           0.0         0.0
1               0.0           0.0         0.0
2          150000.0           0.0    150000.0
3           10000.0           0.0     10000.0
4               0.0           0.0         0.0
5               0.0           0.0         0.0
6               0.0           0.0         0.0
7               0.0           0.0         0.0
8               0.0           0.0         0.0
9               0.0           0.0         0.0
10              0.0           0.0         0.0
11              0.0           0.0         0.0
12              0.0           0.0         0.0
13              0.0           0.0         0.0
14              0.0           0.0         0.0
15              0.0           0.0         0.0
16              0.0           0.0         0.0
17          75000.0           0.0     75000.0
18         120000.0           0.0    120000.0
19              0.0           0.0         0.0
       DAMAGE_PROPERTY  DAMAGE_CRO

In [74]:
# Ensure time is in HHMM format and handle cases where it may be missing
def format_time(time):
    time = str(time).zfill(4)  # Ensures it's always 4 digits (e.g., 30 -> 0030)
    return f"{time[:2]}:{time[2:]}"  # Converts "HHMM" to "HH:MM"

# Convert BEGIN components to datetime
storm_data['BEGIN'] = pd.to_datetime(
    storm_data['BEGIN_YEARMONTH'].astype(str) + storm_data['BEGIN_DAY'].astype(str).str.zfill(2) + " " +
    storm_data['BEGIN_TIME'].apply(format_time),
    format='%Y%m%d %H:%M'
)

# Convert END components to datetime
storm_data['END'] = pd.to_datetime(
    storm_data['END_YEARMONTH'].astype(str) + storm_data['END_DAY'].astype(str).str.zfill(2) + " " +
    storm_data['END_TIME'].apply(format_time),
    format='%Y%m%d %H:%M'
)

# Compute DURATION in hours (decimal format)
storm_data['DURATION_HOURS'] = (storm_data['END'] - storm_data['BEGIN']).dt.total_seconds() / 3600
storm_data['DURATION_MINS'] = (storm_data['END'] - storm_data['BEGIN']).dt.total_seconds() / 60

In [75]:
clean_storm_data = storm_data.copy()
clean_storm_data = clean_storm_data[['NET_DAMAGE', 'DURATION_HOURS', 'DURATION_MINS', 'TOR_F_SCALE', 'TOR_WIDTH', 'TOR_LENGTH', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT']]

In [76]:
clean_storm_data.head()

,NET_DAMAGE,DURATION_HOURS,DURATION_MINS,TOR_F_SCALE,TOR_WIDTH,TOR_LENGTH,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT
0,0.00,0.00,0.00,NaN,NaN,NaN,0,0,0,0
1,0.00,59.85,"3,591.00",NaN,NaN,NaN,0,0,0,0
2,"150,000.00",0.38,23.00,EF1,400.00,6.70,0,0,0,0
3,"10,000.00",0.00,0.00,NaN,NaN,NaN,0,0,0,0
4,0.00,7.00,420.00,NaN,NaN,NaN,0,0,0,0


In [77]:
pd.options.display.float_format = '{:,.2f}'.format
clean_storm_data.describe()

,NET_DAMAGE,DURATION_HOURS,DURATION_MINS,TOR_WIDTH,TOR_LENGTH,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT
count,"1,862,531.00","1,862,531.00","1,862,531.00","197,939.00","197,939.00","1,862,531.00","1,862,531.00","1,862,531.00","1,862,531.00"
mean,"351,488.84",38.28,"2,297.03",39.59,0.86,0.06,0.01,0.01,0.00
std,"30,943,938.87",147.53,"8,851.82",145.94,8.91,2.62,1.87,0.57,0.09
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,0.00,0.05,3.00,0.00,0.00,0.00,0.00,0.00,0.00
75%,0.00,9.00,540.00,20.00,0.10,0.00,0.00,0.00,0.00
max,"17,900,000,000.00","8,828.00","529,680.00","4,576.00","2,315.00","1,150.00","2,400.00",638.00,56.00


In [78]:
storm_data.to_csv("storm_data.csv", index=False)  # Save as CSV
from google.colab import files; files.download("storm_data.csv")  # Download file

clean_storm_data.to_csv("clean_storm_data.csv", index=False)  # Save as CSV
from google.colab import files; files.download("clean_storm_data.csv")  # Download file

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>